In [98]:
import csv
import sqlite3
import gzip
import json
import numpy as np
import pandas as pd

def db_check(name):
    conn = sqlite3.connect(name)
    c = conn.cursor()

    query = "SELECT name, sql FROM sqlite_master WHERE type='table'"
    c.execute(query)
    result = c.fetchall()

    for table_name, schema in result:
        print("---")
        row_count_query = f"SELECT COUNT(*) FROM {table_name}"
        c.execute(row_count_query)
        row_count = c.fetchone()[0]
        print(f"Table '{table_name}': {row_count} rows")
        print(f"Schema:\n{schema}\n")

    conn.close()

def db_clean(name):
    conn = sqlite3.connect(name)
    c = conn.cursor()
    c.execute("VACUUM")
    conn.close()

In [99]:
db_check('project.db')

---
Table 'Authors': 107372 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 427314 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Users': 10000 rows
Schema:
CREATE TABLE Users (
                                        user_id text NOT NULL PRIMARY KEY,
                                        id text,
                                        pw text,
       

In [100]:
conn = sqlite3.connect('project.db')
cursor = conn.cursor()


cursor.execute('ALTER TABLE Reviews RENAME TO Reviews_temp')

cursor.execute('''CREATE TABLE Reviews (review_id text NOT NULL PRIMARY KEY,
                                        review_text text NOT NULL)''')
cursor.execute('INSERT INTO Reviews SELECT review_id, review_text FROM Reviews_temp')

cursor.execute('''CREATE TABLE Ratings (rating_id text NOT NULL PRIMARY KEY,
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        rating INTEGER NOT NULL,
                                        review_id text,
                                        FOREIGN KEY (user_id) REFERENCES Users(user_id),
                                        FOREIGN KEY (book_id) REFERENCES Books(book_id),
                                        FOREIGN KEY (review_id) REFERENCES Reviews(review_id))''')

cursor.execute('INSERT INTO Ratings SELECT review_id, user_id, book_id, rating, review_id FROM Reviews_temp')

cursor.execute('DROP TABLE Reviews_temp')

conn.commit()
cursor.close()
conn.close()
db_check('project.db')

---
Table 'Authors': 107372 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 427314 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Users': 10000 rows
Schema:
CREATE TABLE Users (
                                        user_id text NOT NULL PRIMARY KEY,
                                        id text,
                                        pw text,
       

In [101]:
conn = sqlite3.connect('project.db')
c = conn.cursor()

query = "SELECT user_id FROM Users"
c.execute(query)

userid = c.fetchall()

create_table_query = f'''CREATE TABLE Users_temp (
                                        user_id text NOT NULL PRIMARY KEY,
                                        id text NOT NULL ,
                                        pw text NOT NULL,
                                        nickname text NOT NULL,
                                        email text NOT NULL,
                                        address text NOT NULL,
                                        address_gu text NOT NULL,
                                        address_dong text NOT NULL,
                                        lat_long text NOT NULL,
                                        lat real NOT NULL,
                                        long real NOT NULL
                                        )'''

c.execute(create_table_query)
column_names = ['user_id','id','pw', 'email', 'nickname', 'address', 'address_gu', 'address_dong',\
                'lat_long', 'lat', 'long']


table_name = 'Users_temp'

with open("users.csv", "r") as file:
    csv_reader = csv.reader(file)
    
    next(csv_reader)  
    count = 0
    for row in csv_reader:
        newrow = list(userid[count]) + row[2:]
        cols = ",".join(col for col in column_names)
        insert_query = f"INSERT INTO {table_name} ( {cols} ) VALUES ({','.join(['?'] * len(column_names))})"
        c.execute(insert_query, newrow)
        count += 1

c.execute('DROP TABLE Users')
c.execute('ALTER TABLE Users_temp RENAME TO Users')
conn.commit()
c.close()
conn.close()
db_check('project.db')

---
Table 'Authors': 107372 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 427314 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Reviews': 3374600 rows
Schema:
CREATE TABLE Reviews (review_id text NOT NULL PRIMARY KEY,
                                        review_text text NOT NULL)

---
Table 'Ratings': 3374600 rows
Schema:
CREATE TABLE Ratings (rating

In [102]:
import random
conn = sqlite3.connect('project.db')
c = conn.cursor()

query = "SELECT user_id from Users"
c.execute(query)
userids = c.fetchall()

query = "SELECT book_id from Books"
c.execute(query)
bookids = c.fetchall()

c.execute('''CREATE TABLE Sharings (share_id text NOT NULL PRIMARY KEY,
                                        user_id text NOT NULL,
                                        book_id text NOT NULL,
                                        FOREIGN KEY (user_id) REFERENCES Users(user_id),
                                        FOREIGN KEY (book_id) REFERENCES Books(book_id))''')

column_names = ['share_id', 'user_id', 'book_id']
for i in range(30000):
    rand_user = random.randint(0, len(userids)-1)
    user = list(userids[rand_user])

    rand_book = random.randint(0, len(bookids)-1)
    book = list(bookids[rand_book])
    
    row = [str(i+1)] + user + book
    cols = ",".join(col for col in column_names)
    insert_query = f"INSERT INTO Sharings ( {cols} ) VALUES ({','.join(['?'] * len(column_names))})"
    c.execute(insert_query, row)

conn.commit()
c.close()
conn.close()
db_check('project.db')

---
Table 'Authors': 107372 rows
Schema:
CREATE TABLE Authors (
                                        author_id text NOT NULL PRIMARY KEY,
                                        name text NOT NULL)

---
Table 'Books': 427314 rows
Schema:
CREATE TABLE Books (
                                        book_id text NOT NULL PRIMARY KEY,
                                        title text NOT NULL,
                                        author_id text NOT NULL,
                                        image_url text NOT NULL,
                                        description text NOT NULL,
                                        average_rating real NOT NULL,
                                        FOREIGN KEY (author_id) REFERENCES Authors(author_id))

---
Table 'Reviews': 3374600 rows
Schema:
CREATE TABLE Reviews (review_id text NOT NULL PRIMARY KEY,
                                        review_text text NOT NULL)

---
Table 'Ratings': 3374600 rows
Schema:
CREATE TABLE Ratings (rating

In [103]:
db_clean('project.db')

In [106]:
for table in ['Users', 'Books', 'Authors', 'Ratings', 'Reviews', 'Sharings']:
    conn = sqlite3.connect('project.db')
    cursor = conn.cursor()

    query = f"SELECT * FROM {table} LIMIT 2"
    cursor.execute(query)
    
    print(f'#### {table}')
    rows = cursor.fetchall()
    for row in rows:
        print(row)

    conn.close()

#### Users
('0000c3d51aa099745e93a4e99c4856c8', '1', '1', 'Judith1', '72fb0d00@bkms.com', '서울특별시 강남구 도곡동 도곡대림', '강남구', '도곡동', '[37.4865450724295, 127.039889051865]', 37.4865450724295, 127.039889051865)
('0005f52944ea1992e95d61f287acaea9', '163', '163', 'Dimetry163', '854c0f87@bkms.com', '서울특별시 동작구 상도동 상도래미안1차제2', '동작구', '상도동', '[37.50571609361137, 126.93334302462937]', 37.50571609361137, 126.93334302462937)
#### Books
('7327624', 'The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)', '10333', 'https://images.gr-assets.com/books/1304100136m/7327624.jpg', 'Omnibus book club edition containing the Ladies of Madrigyn and the Witches of Wenshar.', 4.03)
('6066819', 'Best Friends Forever', '9212', 'https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png', "Addie Downs and Valerie Adler were eight when they first met and decided to be best friends forever. But, in the wake of tragedy and betrayal during their teenage years, everything changed. Val went on to f